 #Detección de caras con webcam usando deepface

Deepface crea directorios para descargar los modelos, cuidado si tienes el disco bastante lleno. Es posible configurar la ruta, en mi caso uso  E:\RUNNERS_code\code\DeepFace, tras definir la ruta a través de la variable de entorno DEEPFACE_HOME

In [ ]:
import cv2
from deepface import DeepFace

# documentation https://github.com/serengil/deepface/blob/master/deepface/modules/detection.py
# detector_backends deepface options: 'opencv', 'retinaface', 'mtcnn', 'ssd', 'dlib', 'mediapipe', 'yolov8', 'centerface' or 'skip'
detectors = ['opencv', 'mtcnn', 'retinaface', 'ssd']
detector_idx = 0

# Webcam connection
cap = cv2.VideoCapture(0)
while True:
    # Read frame
    ret, frame = cap.read()
    if not ret:
        break

    try:
        # Face detection
        faces = DeepFace.extract_faces(frame, detector_backend=detectors[detector_idx])
        #print(faces)

        # Draw face bounding box and eyes locations
        for face in faces:
            x, y, w, h = face['facial_area']['x'], face['facial_area']['y'], face['facial_area']['w'], face['facial_area']['h']
            cv2.rectangle(frame, (x, y), (x + w, y + h), (200, 255, 200), 2)  # Dibuja el rectángulo
            if face['facial_area']['left_eye'] is not None:
                cv2.circle(frame, (face['facial_area']['left_eye'][0],face['facial_area']['left_eye'][1]), 3, (0, 255, 0), 2)
                cv2.circle(frame, (face['facial_area']['right_eye'][0],face['facial_area']['right_eye'][1]), 3, (0, 0, 255), 2)

    except Exception as e:
        print("Error:", e)

    # Mostrar el frame
    cv2.putText(frame, f"Detector: {detectors[detector_idx]}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
    cv2.imshow("Video", frame)

    # Salir si se presiona Esc o cambia el detectr
    key = cv2.waitKey(1) & 0xFF
    if key == 27:  # Salir si se presiona 'q'
        break
    elif key == ord('d'):  # Cambiar detector si se presiona 'c'
        detector_idx = (detector_idx + 1) % len(detectors)
        print(f"Cambiado a detector: {detectors[detector_idx]}")

# Liberar la captura y cerrar ventanas
cap.release()
cv2.destroyAllWindows()


Deepface y Retinaface únicamente

In [ ]:
import cv2
import numpy as np
from retinaface import RetinaFace

# Load the mask image (ensure it has an alpha channel)
mask_img = cv2.imread('mask.png', -1)
# Remove the flip unless necessary
mask_img = cv2.flip(mask_img, 0)

# Connect to the webcam
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    cap = cv2.VideoCapture(1)
    if not cap.isOpened():
        print('Error de cámara')
        exit(0)
    else:
        print('Cámara 1')
else:
    print('Cámara 0')

# Set camera resolution
cap.set(3, 640)
cap.set(4, 480)

while True:
    # Get the frame
    ret, frame = cap.read()

    # Detect faces
    faces = RetinaFace.detect_faces(frame)
    if len(faces) > 0:
        for key in faces.keys():
            face = faces[key]
            landmarks = face['landmarks']

            # Get positions of eyes, nose, and mouth
            left_eye = landmarks["left_eye"]
            right_eye = landmarks["right_eye"]
            nose = landmarks["nose"]
            mouth_left = landmarks["mouth_left"]
            mouth_right = landmarks["mouth_right"]
            facial_area = face['facial_area']

            # Draw the reference points
            cv2.circle(frame, (int(left_eye[0]), int(left_eye[1])), 3, (0, 255, 0), -1)
            cv2.circle(frame, (int(right_eye[0]), int(right_eye[1])), 3, (0, 255, 0), -1)
            cv2.circle(frame, (int(nose[0]), int(nose[1])), 3, (0, 255, 0), -1)
            cv2.circle(frame, (int(mouth_left[0]), int(mouth_left[1])), 3, (0, 255, 0), -1)
            cv2.circle(frame, (int(mouth_right[0]), int(mouth_right[1])), 3, (0, 255, 0), -1)
            cv2.circle(frame, (int(facial_area[0]), int(facial_area[1])), 3, (0, 255, 0), -1)

            # Corrected rectangle drawing
            cv2.rectangle(frame, (int(facial_area[0]), int(facial_area[1])),
                                 (int(facial_area[2]), int(facial_area[3])), (0, 255, 0), 2)

            # Calculate the angle between the eyes (in degrees)
            delta_x = right_eye[0] - left_eye[0]
            delta_y = right_eye[1] - left_eye[1]
            angle = np.degrees(np.arctan2(delta_y, delta_x))

            # Calculate the center between the eyes
            eyes_center = ((left_eye[0] + right_eye[0]) / 2,
                           (left_eye[1] + right_eye[1]) / 2)

            # Calculate face width based on the distance between the mouth corners
            face_width = np.linalg.norm(np.array(mouth_right) - np.array(mouth_left))

            # Adjust mask size to match the face width
            mask_width = int(face_width * 1.2)  # Adjust scaling factor as needed
            mask_height = int(mask_width * mask_img.shape[0] / mask_img.shape[1])  # Maintain aspect ratio

            # Ensure mask dimensions are valid
            if mask_width > 0 and mask_height > 0:
                # Resize the mask image
                resized_mask = cv2.resize(mask_img, (mask_width, mask_height), interpolation=cv2.INTER_AREA)

                # Rotate the mask image around its center
                M = cv2.getRotationMatrix2D((mask_width / 2, mask_height / 2), angle, 1)
                rotated_mask = cv2.warpAffine(resized_mask, M, (mask_width, mask_height), flags=cv2.INTER_AREA, borderMode=cv2.BORDER_CONSTANT, borderValue=(0,0,0,0))

                # Calculate the position to overlay the mask
                x1 = int(eyes_center[0] - mask_width / 2)
                y1 = int(eyes_center[1] - mask_height / 3)  # Adjust the vertical position as needed
                x2 = x1 + mask_width
                y2 = y1 + mask_height

                # Adjust coordinates to fit within the frame
                x1_mask = 0
                y1_mask = 0
                x2_mask = mask_width
                y2_mask = mask_height

                if x1 < 0:
                    x1_mask = -x1
                    x1 = 0
                if y1 < 0:
                    y1_mask = -y1
                    y1 = 0
                if x2 > frame.shape[1]:
                    x2_mask = mask_width - (x2 - frame.shape[1])
                    x2 = frame.shape[1]
                if y2 > frame.shape[0]:
                    y2_mask = mask_height - (y2 - frame.shape[0])
                    y2 = frame.shape[0]

                # Ensure integer values
                x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])
                x1_mask, y1_mask, x2_mask, y2_mask = map(int, [x1_mask, y1_mask, x2_mask, y2_mask])

                # Extract the mask region
                mask_to_overlay = rotated_mask[y1_mask:y2_mask, x1_mask:x2_mask]

                # Check if the mask has an alpha channel
                if mask_to_overlay.shape[2] == 4:
                    alpha_mask = mask_to_overlay[:, :, 3] / 255.0
                    alpha_inv = 1.0 - alpha_mask
                    mask_rgb = mask_to_overlay[:, :, :3]

                    # Get the region of interest from the frame
                    roi = frame[y1:y2, x1:x2]

                    # Combine the mask with the ROI
                    for c in range(0, 3):
                        roi[:, :, c] = (alpha_mask * mask_rgb[:, :, c] +
                                        alpha_inv * roi[:, :, c])

                    # Place the combined ROI back into the frame
                    frame[y1:y2, x1:x2] = roi
                else:
                    print("La imagen de la máscara no tiene un canal alfa.")
            else:
                print("Dimensiones de la máscara no válidas.")

    # Show the resulting frame
    cv2.imshow('Cam', frame)

    # Exit loop when 'Esc' key is pressed
    if cv2.waitKey(1) & 0xFF == 27:
        break

# Release the capture and close windows
cap.release()
cv2.destroyAllWindows()

In [ ]:
import cv2
import time
import numpy as np
from retinaface import RetinaFace

# Load the mask image
mask_img = cv2.imread('mask.png', -1)  # Ensure this path is correct

# Webcam connection
cap = cv2.VideoCapture(0)
# ... [Your existing camera setup code] ...

while True:
    # Get frame
    ret, frame = cap.read()

    # Detect faces
    faces = RetinaFace.detect_faces(frame)
    if len(faces) > 0:
        for idx in range(1, len(faces)+1):
            id = 'face_' + str(idx)
            landmarks = faces[id]['landmarks']

            # Get positions of the eyes
            left_eye = landmarks["left_eye"]
            right_eye = landmarks["right_eye"]

            # Calculate the center between eyes
            eye_center = (
                int((left_eye[0] + right_eye[0]) / 2),
                int((left_eye[1] + right_eye[1]) / 2)
            )

            # Calculate angle between the eyes
            delta_x = right_eye[0] - left_eye[0]
            delta_y = right_eye[1] - left_eye[1]
            angle = np.degrees(np.arctan2(delta_y, delta_x))

            # Calculate the distance between the eyes
            eye_distance = np.hypot(delta_x, delta_y)

            # Determine the size of the mask
            mask_width = int(eye_distance * 2)  # Adjust scaling factor as needed
            mask_height = int(mask_width * mask_img.shape[0] / mask_img.shape[1])

            # Resize the mask image
            resized_mask = cv2.resize(mask_img, (mask_width, mask_height), interpolation=cv2.INTER_AREA)

            # Rotate the mask image
            M = cv2.getRotationMatrix2D((mask_width / 2, mask_height / 2), angle, 1)
            rotated_mask = cv2.warpAffine(resized_mask, M, (mask_width, mask_height), flags=cv2.INTER_AREA)

            # Calculate position to overlay the mask
            x1 = int(eye_center[0] - mask_width / 2)
            y1 = int(eye_center[1] - mask_height / 2)

            # Ensure the coordinates are within the frame
            x1 = max(0, x1)
            y1 = max(0, y1)
            x2 = min(frame.shape[1], x1 + mask_width)
            y2 = min(frame.shape[0], y1 + mask_height)

            # Adjust the mask size if it goes beyond the frame
            rotated_mask = rotated_mask[0:(y2 - y1), 0:(x2 - x1)]

            # Extract the alpha mask and BGR channels from the mask image
            if rotated_mask.shape[2] == 4:
                alpha_mask = rotated_mask[:, :, 3] / 255.0
                alpha_inv = 1.0 - alpha_mask
                mask_bgr = rotated_mask[:, :, :3]

                # Get the region of interest from the frame
                roi = frame[y1:y2, x1:x2]

                # Blend the mask with the ROI
                for c in range(0, 3):
                    roi[:, :, c] = (alpha_mask * mask_bgr[:, :, c] + alpha_inv * roi[:, :, c])

                # Place the blended ROI back into the frame
                frame[y1:y2, x1:x2] = roi
            else:
                print("Mask image does not have an alpha channel.")

    # Display the resulting frame
    cv2.imshow('Cam', frame)

    # Break the loop on 'Esc' key press
    if cv2.waitKey(1) & 0xFF == 27:
        break

# Release the capture and close windows
cap.release()
cv2.destroyAllWindows()

In [ ]:
import cv2
import numpy as np
from retinaface import RetinaFace
from deepface import DeepFace

# Load visual effects images
effects = {
    'happy': cv2.imread('sunglasses.png', -1),
    'sad': cv2.imread('rain.png', -1),
    'surprise': cv2.imread('wow.png', -1),
    'angry': cv2.imread('flames.png', -1),
    # Add more emotions and corresponding images as needed
}

# Webcam connection
cap = cv2.VideoCapture(0)

while True:
    # Get frame
    ret, frame = cap.read()
    if not ret:
        break

    # Detect faces
    faces = RetinaFace.detect_faces(frame)
    if faces:
        for idx, (key, face_data) in enumerate(faces.items()):
            # Extract facial region
            facial_area = face_data['facial_area']
            x, y, w, h = facial_area[0], facial_area[1], facial_area[2]-facial_area[0], facial_area[3]-facial_area[1]
            face_img = frame[y:y+h, x:x+w]

            # Emotion recognition
            try:
                analysis = DeepFace.analyze(face_img, actions=['emotion'], enforce_detection=False)
                emotion = analysis['dominant_emotion']
            except:
                emotion = 'neutral'

            # Select effect based on emotion
            effect_img = effects.get(emotion)
            if effect_img is not None:
                # Resize and position the effect
                effect_resized = cv2.resize(effect_img, (w, h))
                alpha_s = effect_resized[:, :, 3] / 255.0
                alpha_l = 1.0 - alpha_s

                for c in range(0, 3):
                    frame[y:y+h, x:x+w, c] = (alpha_s * effect_resized[:, :, c] +
                                              alpha_l * frame[y:y+h, x:x+w, c])
            else:
                # Optional: handle neutral or unrecognized emotions
                pass

    # Display the resulting frame
    cv2.imshow('Emotion-Responsive Filters', frame)

    # Break the loop on 'Esc' key press
    if cv2.waitKey(1) & 0xFF == 27:
        break

# Release the capture and close windows
cap.release()
cv2.destroyAllWindows()

In [ ]:
import cv2
import numpy as np
from retinaface import RetinaFace

# Webcam connection
cap = cv2.VideoCapture(0)

def add_sparkles(frame, center, num_sparkles=10, color=(0, 255, 255)):
    """Add sparkles around a given center point on the frame."""
    for _ in range(num_sparkles):
        offset_x = np.random.randint(-30, 30)
        offset_y = np.random.randint(-30, 30)
        cv2.circle(frame, (center[0] + offset_x, center[1] + offset_y), 2, color, -1)

def enlarge_eyes(frame, eye_center, size=1.5):
    """Enlarge eye regions to create a surprised look."""
    x, y = eye_center
    w, h = 30, 20  # Width and height around the eye center
    roi = frame[y-h:y+h, x-w:x+w]
    if roi.shape[0] > 0 and roi.shape[1] > 0:  # Check if ROI is within bounds
        enlarged_eye = cv2.resize(roi, None, fx=size, fy=size, interpolation=cv2.INTER_LINEAR)
        frame[y-h:y+h, x-w:x+w] = enlarged_eye[:h*2, :w*2]  # Place resized eye back in frame

def apply_red_tint(frame):
    """Apply a red tint to the frame."""
    frame[:, :, 2] = cv2.add(frame[:, :, 2], 50)

def add_tears(frame, eye_center):
    """Draw tears below the eye to simulate a sad expression."""
    x, y = eye_center
    for i in range(5):
        cv2.circle(frame, (x, y + 15 + i*10), 5, (255, 0, 0), -1)  # Draw blue tear circles

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Detect faces and get landmarks
    faces = RetinaFace.detect_faces(frame)
    if faces:
        for face in faces.values():
            landmarks = face['landmarks']
            print(landmarks)
            left_eye = np.array(landmarks['left_eye'], np.int32)
            right_eye = np.array(landmarks['right_eye'], np.int32)
            mouth_left = np.array(landmarks['mouth_left'], np.int32)
            mouth_right = np.array(landmarks['mouth_right'], np.int32)
            # mouth_top = np.array(landmarks['mouth_up'], np.int32)
            # mouth_bottom = np.array(landmarks['mouth_down'], np.int32)
            
            # Calculate mouth aspect ratio (MAR) to detect smile
            mar = np.linalg.norm(mouth_left - mouth_right) / np.linalg.norm(left_eye - right_eye)

            # Calculate eye aspect ratio (EAR) to detect surprise
            eye_aspect_ratio = np.linalg.norm(left_eye - right_eye)

            # Detect expressions and apply effects
            if True:  # Happy expression detected
                center_mouth = tuple((mouth_left + mouth_right) // 2)
                add_sparkles(frame, center_mouth)
                cv2.putText(frame, 'Happy', (30, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

            elif True:  # Surprised expression detected
                enlarge_eyes(frame, left_eye)
                enlarge_eyes(frame, right_eye)
                cv2.putText(frame, 'Surprised', (30, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

            elif True:  # Angry expression detected (small MAR indicates tight lips)
                apply_red_tint(frame)
                cv2.putText(frame, 'Angry', (30, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

            elif True:  # Sad expression detected
                add_tears(frame, left_eye)
                add_tears(frame, right_eye)
                cv2.putText(frame, 'Sad', (30, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

            else:  # Neutral expression, apply grayscale filter
                gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                frame = cv2.merge([gray_frame, gray_frame, gray_frame])
                cv2.putText(frame, 'Neutral', (30, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    # Display the resulting frame
    cv2.imshow('Real-Time Expression Manipulation', frame)

    # Break the loop on 'Esc' key press
    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()

In [5]:
import cv2
import numpy as np
import time
from retinaface import RetinaFace
from deepface import DeepFace  # Added DeepFace import

# Debug mode
debug = True  # Set to False to disable debug mode

# Load filter images with alpha channel
glasses = cv2.imread('gafas.png', cv2.IMREAD_UNCHANGED)
mustache = cv2.imread('bigote.png', cv2.IMREAD_UNCHANGED)
hat = cv2.imread('sombrero.png', cv2.IMREAD_UNCHANGED)

# Verify if filter images are loaded correctly
if glasses is None or mustache is None or hat is None:
    print("Error loading filter images.")
    exit()

# Connect to the webcam
cap = cv2.VideoCapture(0)

# Initialize variables for FPS calculation
fps = 0
frame_count = 0
start_time = time.time()

# Initialize variables for face detection frequency
detection_interval = 3  # Detect faces every 3 frames
emotion_interval = 5  # Analyze emotions every 5 frames
faces = None
emotion_results = {}

# Flags to indicate which filters are active
use_glasses = True
use_mustache = True
use_hat = True

def overlay_image(frame, overlay, position):
    """Overlay an image with transparency over the frame."""
    x, y = position
    h, w = overlay.shape[:2]

    # Ensure the coordinates are within the frame
    if x < 0 or y < 0 or x + w > frame.shape[1] or y + h > frame.shape[0]:
        return

    # Get region of interest on the frame
    roi = frame[y:y+h, x:x+w]

    # Separate the color and alpha channels
    overlay_img = overlay[:, :, :3]
    overlay_mask = overlay[:, :, 3]

    # Convert mask to 3 channels and normalize
    overlay_mask = cv2.cvtColor(overlay_mask, cv2.COLOR_GRAY2BGR) / 255.0
    background_mask = 1.0 - overlay_mask

    # Blend the overlay with the ROI
    roi = (overlay_img * overlay_mask + roi * background_mask).astype(np.uint8)

    # Put the blended image back into the frame
    frame[y:y+h, x:x+w] = roi

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Flip the frame horizontally
    frame = cv2.flip(frame, 1)

    # Increment frame count
    frame_count += 1

    # Calculate FPS every 10 frames
    if frame_count % 10 == 0:
        end_time = time.time()
        fps = 10 / (end_time - start_time)
        start_time = end_time

    if debug:
        # Display FPS on the frame
        cv2.putText(frame, f'FPS: {fps:.2f}', (10, 20),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

    # Detect faces every 'detection_interval' frames
    if frame_count % detection_interval == 0:
        faces = RetinaFace.detect_faces(frame)

    # Analyze emotions every 'emotion_interval' frames
    if frame_count % emotion_interval == 0:
        analyze_emotions = True
    else:
        analyze_emotions = False

    face_count = 0
    if isinstance(faces, dict):
        face_count = len(faces)
        for face_id, face in faces.items():
            facial_area = face['facial_area']
            landmarks = face['landmarks']
            x1, y1, x2, y2 = facial_area
            face_roi = frame[y1:y2, x1:x2]

            # Perform emotion analysis
            if analyze_emotions:
                try:
                    analysis = DeepFace.analyze(face_roi, actions=['emotion'], enforce_detection=False)
                    
                    if isinstance(analysis, list):
                        analysis = analysis[0]
                        
                    emotion = analysis['dominant_emotion']
                    
                except Exception as e:
                    print(f"Emotion analysis failed: {e}")
                    emotion_results[face_id] = 'Unknown'
            else:
                emotion = emotion_results.get(face_id, 'Analyzing...')

            if debug:
                # Display the emotion label on the frame
                cv2.putText(frame, f'Emotion: {emotion}', (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

                # Draw bounding rectangle
                face_width = x2 - x1
                face_height = y2 - y1
                cv2.rectangle(frame, (x1, y1), (x2, y2),
                              (255, 0, 0), 2)
                cv2.putText(frame, f'Face: ({x1},{y1}) ({x2},{y2}) W:{face_width} H:{face_height}',
                            (x1, y2 + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 0, 0), 1)

                # Display landmarks with names and coordinates
                for point_name, point in landmarks.items():
                    x, y = int(point[0]), int(point[1])
                    cv2.circle(frame, (x, y), 2, (0, 255, 0), -1)
                    cv2.putText(frame, f'{point_name} ({x},{y})', (x + 5, y - 5),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 255, 0), 1)

            # Calculate positions and sizes for filters
            left_eye = landmarks['left_eye']
            right_eye = landmarks['right_eye']
            nose = landmarks['nose']
            mouth_left = landmarks['mouth_left']
            mouth_right = landmarks['mouth_right']

            eye_center = ((left_eye[0] + right_eye[0]) / 2,
                          (left_eye[1] + right_eye[1]) / 2)
            eye_width = int(abs(right_eye[0] - left_eye[0]) * 2)
            eye_height = int(eye_width * glasses.shape[0] / glasses.shape[1])
            eye_x = int(eye_center[0] - eye_width / 2)
            eye_y = int(eye_center[1] - eye_height / 2)

            # Adjust position and size of mustache
            mouth_center = ((mouth_left[0] + mouth_right[0]) / 2,
                            (mouth_left[1] + mouth_right[1]) / 2)
            mustache_width = int(abs(mouth_right[0] - mouth_left[0]) * 1.5)
            mustache_height = int(
                mustache_width * mustache.shape[0] / mustache.shape[1])
            mustache_x = int(mouth_center[0] - mustache_width / 2)
            mustache_y = int(mouth_center[1] - mustache_height / 1.4)

            # Calculate position and size of hat
            head_width = int(eye_width * 1.5)
            head_height = int(
                head_width * hat.shape[0] / hat.shape[1])
            head_x = int(eye_center[0] - head_width / 2)
            head_y = int(eye_y - head_height + eye_height // 2)

            # Resize and overlay filters based on user selection
            if use_glasses:
                glasses_resized = cv2.resize(
                    glasses, (eye_width, eye_height), interpolation=cv2.INTER_AREA)
                overlay_image(frame, glasses_resized, (eye_x, eye_y))

            if use_mustache:
                mustache_resized = cv2.resize(
                    mustache, (mustache_width, mustache_height), interpolation=cv2.INTER_AREA)
                overlay_image(frame, mustache_resized, (mustache_x, mustache_y))

            if use_hat:
                hat_resized = cv2.resize(
                    hat, (head_width, head_height), interpolation=cv2.INTER_AREA)
                overlay_image(frame, hat_resized, (head_x, head_y))

    if debug:
        # Display number of faces detected
        cv2.putText(frame, f'Faces detected: {face_count}', (10, 40),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

    # Show the result
    cv2.imshow('Fun Filter', frame)

    # Handle key events
    key = cv2.waitKey(1) & 0xFF
    if key == 27:
        # Exit with 'Esc' key
        break
    elif key == ord('d'):
        # Toggle debug mode with 'd' key
        debug = not debug
    elif key == ord('g'):
        # Toggle glasses filter with 'g' key
        use_glasses = not use_glasses
    elif key == ord('m'):
        # Toggle mustache filter with 'm' key
        use_mustache = not use_mustache
    elif key == ord('h'):
        # Toggle hat filter with 'h' key
        use_hat = not use_hat

cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 

In [ ]:
import cv2
import numpy as np
import time
from deepface import DeepFace
from retinaface import RetinaFace
import random

# Debug mode
debug = True  # Set to False to disable debug mode

# Load funny images with alpha channel
funny_images = [
    cv2.imread('balloon.png', cv2.IMREAD_UNCHANGED),
    cv2.imread('confetti.png', cv2.IMREAD_UNCHANGED),
    cv2.imread('laugh_emoji.png', cv2.IMREAD_UNCHANGED)
]

# Verify if funny images are loaded correctly
for idx, img in enumerate(funny_images):
    if img is None:
        print(f"Error loading image {idx}")
        exit()

# Connect to the webcam
cap = cv2.VideoCapture(0)

# Initialize variables for FPS calculation
fps = 0
frame_count = 0
start_time = time.time()

# Initialize variables for animated images
animated_images = []

def overlay_image(background, overlay, position):
    """Overlay an image with transparency over the background."""
    x, y = position
    h, w = overlay.shape[:2]

    # Ensure the coordinates are within the frame
    if x < 0 or y < 0 or x + w > background.shape[1] or y + h > background.shape[0]:
        return

    # Get region of interest
    roi = background[y:y+h, x:x+w]

    # Separate channels
    overlay_img = overlay[:, :, :3]
    overlay_mask = overlay[:, :, 3:] / 255.0

    background_mask = 1.0 - overlay_mask

    # Blend the images
    roi = (overlay_img * overlay_mask + roi * background_mask).astype(np.uint8)

    background[y:y+h, x:x+w] = roi

# Initialize background subtractor for person contour detection
backSub = cv2.createBackgroundSubtractorMOG2()

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Flip the frame horizontally
    frame = cv2.flip(frame, 1)

    # Increment frame count
    frame_count += 1

    # Calculate FPS every 10 frames
    if frame_count % 10 == 0:
        end_time = time.time()
        fps = 10 / (end_time - start_time)
        start_time = end_time

    if debug:
        # Display FPS on the frame
        cv2.putText(frame, f'FPS: {fps:.2f}', (10, 20),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

    # Detect faces
    faces = RetinaFace.detect_faces(frame)
    face_count = 0
    emotion = None

    if isinstance(faces, dict):
        face_count = len(faces)
        for key in faces:
            face = faces[key]
            facial_area = face['facial_area']
            x1, y1, x2, y2 = facial_area

            # Extract face ROI
            face_roi = frame[y1:y2, x1:x2]

            # Emotion detection using DeepFace
            try:
                analysis = DeepFace.analyze(face_roi, actions=['emotion'], enforce_detection=False)
                emotion = analysis['dominant_emotion']
                if debug:
                    cv2.putText(frame, f'Emotion: {emotion}', (x1, y1 - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
            except Exception as e:
                if debug:
                    print(f"Emotion detection error: {e}")

            # Draw bounding box around the face in debug mode
            if debug:
                cv2.rectangle(frame, (x1, y1), (x2, y2),
                              (255, 0, 0), 2)

            # Process only the first detected face
            break

    # Person contour detection using background subtraction
    fg_mask = backSub.apply(frame)
    _, fg_mask = cv2.threshold(fg_mask, 244, 255, cv2.THRESH_BINARY)
    fg_mask = cv2.erode(fg_mask, None, iterations=2)
    fg_mask = cv2.dilate(fg_mask, None, iterations=2)

    # Find contours
    contours, _ = cv2.findContours(fg_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    person_mask = np.zeros_like(frame[:, :, 0])

    if contours:
        # Assume the largest contour is the person
        largest_contour = max(contours, key=cv2.contourArea)
        cv2.drawContours(person_mask, [largest_contour], -1, 255, cv2.FILLED)

    # Invert person mask to get background mask
    background_mask = cv2.bitwise_not(person_mask)

    # Create background where we'll place the animated images
    background = cv2.bitwise_and(frame, frame, mask=background_mask)

    # Update animated images positions
    new_animated_images = []
    for img_info in animated_images:
        img = img_info['image']
        x = img_info['x']
        y = img_info['y'] - img_info['speed']  # Move up by 'speed' pixels

        # If the image is still within the frame, draw it
        if y + img.shape[0] > 0:
            overlay_image(background, img, (x, y))
            img_info['y'] = y
            new_animated_images.append(img_info)
    animated_images = new_animated_images

    # Add new images based on emotion
    if emotion in ['happy', 'surprise', 'neutral'] and frame_count % 30 == 0:
        # Randomly select a funny image
        img = random.choice(funny_images)
        img_h, img_w = img.shape[:2]

        # Random x position within the frame width
        x = random.randint(0, frame.shape[1] - img_w)
        y = frame.shape[0]  # Start from the bottom
        speed = random.randint(2, 5)  # Random speed for variation

        animated_images.append({'image': img, 'x': x, 'y': y, 'speed': speed})

    # Combine background with person
    person = cv2.bitwise_and(frame, frame, mask=person_mask)
    frame_result = cv2.add(background, person)

    if debug:
        # Display number of faces detected
        cv2.putText(frame_result, f'Faces detected: {face_count}', (10, 40),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

    # Show the result
    cv2.imshow('Emotion Filter', frame_result)

    # Handle key events
    key = cv2.waitKey(1) & 0xFF
    if key == 27:
        # Exit with 'Esc' key
        break
    elif key == ord('d'):
        # Toggle debug mode with 'd' key
        debug = not debug

cap.release()
cv2.destroyAllWindows()